In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
    )

    return response.choices[0].message.content

def askJSON(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
def write_string_to_file(filename, content):
    lines = content.split('\n')
    cleaned_lines = [line.lstrip("=-").strip() for line in lines]
    cleaned_content = '\n'.join(cleaned_lines)
    try:
        with open(filename, 'w') as file:
            file.write(cleaned_content)
        print(f"String has been written to {filename}")
    except IOError as e:
        print(f"An error occurred while writing to the file: {e}")

In [3]:
usecase_name = "EditGenre"
project_name = "Online Library"
main_flow_prompt="""
Main scenario:
User clicks the "Update" button (to the right from the genre he/she wants to update) on the genre list/details page;
Application displays form to enter genre data;
User enters genre data he/she wants to update and presses "Submit" button;
If any data is entered incorrectly, incorrect data messages are displayed;
If entered data is valid, then the genre is updated in the database;
If an error occurs, then the error message is displayed;
If a new genre record is successfully updated, then the list of the genres with updated records is displayed.
"""

alt_prompt_1="""
Main scenario:
User clicks the "Update" button (to the right from the genre he/she wants to update) on the genre list/details page;
Application displays form to enter genre data;
User enters genre data he/she wants to update and presses "Submit" button;
If any data is entered incorrectly, incorrect data messages are displayed;
If entered data is valid, then the genre is updated in the database;
If an error occurs, then the error message is displayed;
If a new genre record is successfully updated, then the list of the genres with updated records is displayed.

Cancel operation scenario:
User clicks the "Update" button (to the right from the genre he/she wants to update) on the genre list/details page;
Application displays form to enter genre data;
User may start entering the data into the form fields;
Before pressing "Submit" button user presses “Cancel” button;
Data doesn’t update in the database, then a list of genres records is displayed to the user;
If the user selects the menu item "Books”, "Genres”, "Authors", "Home" or "API", the data will not be update in the database, and the corresponding form with updated data will be opened.
"""

prompt_all="""
Edit the genre
Description
The edit page is designed to edit an existing genre record in the database.

Main scenario:
User clicks the "Update" button (to the right from the genre he/she wants to update) on the genre list/details page;
Application displays form to enter genre data;
User enters genre data he/she wants to update and presses "Submit" button;
If any data is entered incorrectly, incorrect data messages are displayed;
If entered data is valid, then the genre is updated in the database;
If an error occurs, then the error message is displayed;
If a new genre record is successfully updated, then the list of the genres with updated records is displayed.

Cancel operation scenario:
User clicks the "Update" button (to the right from the genre he/she wants to update) on the genre list/details page;
Application displays form to enter genre data;
User may start entering the data into the form fields;
Before pressing "Submit" button user presses “Cancel” button;
Data doesn’t update in the database, then a list of genres records is displayed to the user;
If the user selects the menu item "Books”, "Genres”, "Authors", "Home" or "API", the data will not be update in the database, and the corresponding form with updated data will be opened.

When editing a genre, the following details are entered:
Name
Description
"""

In [4]:
#Prompt
MAIN_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow of a use case.
Then you predict all scenarios that can happen in this flow.

Rules to predict scenarios:
- Stay close to the details described in the flow.
- Choose important cases to generate, important scenario is the scenario that users are more likely to encounter it.
- Limit the appearance of scenarios that are hard to happen. 
- A scenario encompasses a whole function, not just verifying individual steps.
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
I only need scenarios's name for the output, I do not need the steps to go with it.
Example:
The scenario's name structure should start with who or what role can do this action + what they are testing in this scenario (including information about they can or cannot do the action. If they cannot, why).
Users successfully create a new account.
Users cannot create a new account if the username field is blank.
Users can navigate to Add New Lesson Page via Navbar Option.
"""

SUB_FLOW_SYSTEM_PROMPT="""
I want you to act as software tester.
Your task is to read this information about one main flow and one alternative or exception flow of a use case.
Then you predict all scenarios that can lead user from the main flow to change to the alternative or exception flow mentioned for creating test cases.

Rules to predict scenarios:
- A scenario encompasses a whole function, not just verifying individual steps.
- If there is no other action in the flow beside clicking or there is no condition to vary the user's actions, that flow has one scenario only.
- A scenario often refers to a specific sequence of events or user actions that could potentially lead to a change in how the application behaves or responds.
- Test scenarios should be derived from cohesive sequences of steps that represent meaningful user interactions, rather than isolated steps.
- A scenario should cover from the first step to the final step in the flow, the start or the result of the scenario could be different.
- You cannot separate parts of a flow to be a scenario (Example: predict multiple scenarios for a flow by dividing steps into parts) because each scenarios should be independent and require a complete flow to proceed.
- Do not generate scenarios with user analysis. (Example: User accidentally do A and user intentionally do A is the same scenario, so do not consider about "accidentally" or "intentionally" in scenario)
- Do not choose another option that is not chosen by the flow, eventhough it is mentioned (Example: A pop up with OK and Cancel, the flow only has step choose OK. Do not generate scenario that press Cancel)
- Do not generate scenario to test only the main flow.
I only need scenarios's name for the output, I do not need the steps to go with it.
Example:
The scenario's name structure should start with who or what role can do this action + what they are testing in this scenario (including information about they can or cannot do the action. If they cannot, why).
Users successfully create a new account.
Users cannot create a new account if the username field is blank.
Users can navigate to Add New Lesson Page via Navbar Option.
"""

EXTRACT_CONDITION_SYSTEM_PROMPT="""
Given use case flows of a feature.
Your task is to identify all the interactive elements within the feature. 
For each interactive element:
Identify what type of that element (button,buttons, icon,scroll, text field,text area, tab, radio buttons, menu, combobox, sliders, switches, dialog, link, form,rating, filter).
Identify all the conditions mentioned in the use case of that element that would make the element valid and the conditions that would make the element invalid based on the description of the use case flow.
Do not arbitrarily create additional conditions that not mention in the use case flow.
Return the element extracted in json format.
The JSON format should follow the following structure:
{"Name of interactive element": {"condition": {valid:"conditions that make element valid", invalid: "conditions that make element invalid"}, "type": "element type"}}
Examples of output json format template: 
{"Username": {"condition": {"valid": "must be over 8 characters and below 30 characters, must be entered", racter, empty"}, "type": "text field"}}
{"Search button": {"condition": {"valid": "", "invalid": ""}, "type": "text field"}}
"""

GEN_SCENARIO_FOR_CONDITION_PROMPT="""
Given a list of interaction element for input value, their extracted conditions and use c"invalid": "below 8 characters, over 30 chaase specification.
For each given invalid condition:
- Generate a test scenario that test that condition.
Do not generate test scenario to test element/condition that not mention in the given element list.
Do not generate test scenario to test valid conditions.
I only need scenarios's name for the output, I do not need the steps to go with it.
Example:
The scenario's name structure should start with who or what role can do this action + what they are testing in this scenario (including information about they can or cannot do the action. If they cannot, why).
Users successfully create a new account.
Users cannot create a new account if the username field is blank.
Users can navigate to Add New Lesson Page via Navbar Option.
"""


FILTER_SYSTEM_PROMPT="""
Your task is to read all of this scenarios generation from many sources.
Then you remove all the same copies because there are many duplicate scenarios in meanings inside the input.
If a scenario from the flow scenario is duplicate with a scenario from the condition scenario, delete the one from the flow scenario and keep the one from the condition.
Make sure every scenarios in the response is unique.
Return scenario name only.
"""

FILTER_SYSTEM_PROMPT3= """You will be given a list of scenarios group by action.
Your task is to identify all the duplicate scenarios and delete it.
Make sure every scenarios in the response is unique. 
There are also cases that different actions could be the same meaning with the context. Please check through all the actions and their corresponding scenarios to see if two different words are being used to refer a same action.
Return scenario name only.
There are scenarios with different wording but test the same condition.

Example for conditional duplicate:
Create:
1. User cannot create account because of invalid username.
2. User cannot create account because username has more than 8 characters.
3. User cannot create account because username has less than 5 characters.
Scenario 1 is consider to be duplicate with scenario 2 and scenario 3 because it does not make it clear what is invalid, so there is chance for scenario 1 to produce test cases duplicate with 2 and 3. But if 2 and 3 is not exists, scenario 1 is consider to be unique.

Example for same goal:
Exit:
1. User exits the page.
2. User press exits and confirm the exit.
3. User press exits but cancel the exit.
Leave:
4. User leaves the page.
Scenario 1 is consider to be duplicate with scenario 2 and 4, because the goal is the same and the test steps is predicted to be the same for the test cases created after this phase. For scenario 4, even though the verb is different but they are mentioning the same action. But scenario 1 and scenario 3 is different from each other because their final results and steps are expected to be different.

"""

Prompt_Extract_Action = """
Read this list of test scenarios.
For each test scenario:
    Define the action of user.     For example:  "successfully create", "cannot create", "cancels", "navigates". Keep the number of action minimum. 
    For each action: Group the test scenario with that action.
Return the result in json format.
The JSON format should follow the following structure:
{
  "action": [test scenarios],
  }

An example of a test scenario: 
{
  "successfully create": ["user successfully create object"],
  "cannot create": ["user cannot create object with invalid data","user cannot create object with missing data"]
  }
"""

# Identify what type of that element (button,buttons, icon,scroll, text field, radio buttons, menu, menu for navigation,menu for opening dialog or another menu, menu for filter, sliders, switches, dialog, link, form,rating, filter).


In [5]:
promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

filtercontent = ""

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '/n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)
  filtercontent =  "Conditional scenario:" + "\n" + condition_scenario_response


promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)

promptSubScenario1 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_1}
  ]
sub1_gpt_response = ask(promptSubScenario1, client, model)

filtercontent +="\n" + "Flow scenario:" + "\n" + main_gpt_response + "\n" + sub1_gpt_response
print(filtercontent)
promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
print("\n" + "Final result:" + filter_gpt_response)

promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
print("\n" + "Final result:" + filter_gpt_response)

promptExtractCondition = [
    { "role": "system", "content": Prompt_Extract_Action},
    { "role": "user", "content": filter_gpt_response}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
print(gpt_response)

group_testscearnio = json.loads(gpt_response)

promptExtractCondition = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT3},
    { "role": "user", "content": str(group_testscearnio)}
  ]
gpt_response = ask(promptExtractCondition, client, model)
write_string_to_file(usecase_name + "-Filter1.csv", filter_gpt_response)
print(gpt_response)

{'Genre Name': {'condition': {'valid': 'user enters a name for the genre', 'invalid': 'name field is left empty'}, 'type': 'text field'}, 'Genre Description': {'condition': {'valid': 'user enters a description for the genre', 'invalid': 'description field is left empty'}, 'type': 'text area'}}
User cannot update genre if the Genre Name field is left empty.
User cannot update genre if the Genre Description field is left empty.
Conditional scenario:
User cannot update genre if the Genre Name field is left empty.
User cannot update genre if the Genre Description field is left empty.
Flow scenario:
User successfully updates a genre record.
User cannot update a genre record if required fields are blank.
User receives incorrect data messages when entering invalid genre data.
User encounters an error message during the genre update process.
User views updated genre list after successful record update.
Users successfully update a genre record.
Users cannot update a genre record if entered data

In [6]:
promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

filtercontent = ""

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '/n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)
  filtercontent =  "Conditional scenario:" + "\n" + condition_scenario_response


promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)

promptSubScenario1 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_1}
  ]
sub1_gpt_response = ask(promptSubScenario1, client, model)

filtercontent +="\n" + "Flow scenario:" + "\n" + main_gpt_response + "\n" + sub1_gpt_response
print(filtercontent)
promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
print("\n" + "Final result:" + filter_gpt_response)

promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
print("\n" + "Final result:" + filter_gpt_response)

promptExtractCondition = [
    { "role": "system", "content": Prompt_Extract_Action},
    { "role": "user", "content": filter_gpt_response}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
print(gpt_response)

group_testscearnio = json.loads(gpt_response)

promptExtractCondition = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT3},
    { "role": "user", "content": str(group_testscearnio)}
  ]
gpt_response = ask(promptExtractCondition, client, model)
write_string_to_file(usecase_name + "-Filter2.csv", filter_gpt_response)
print(gpt_response)

{'Name': {'condition': {'valid': 'must be entered', 'invalid': 'is empty'}, 'type': 'text field'}, 'Description': {'condition': {'valid': 'must be entered', 'invalid': 'is empty'}, 'type': 'text area'}}
Users cannot update genre if the Name field is empty.
Users cannot update genre if the Description field is empty.
Conditional scenario:
Users cannot update genre if the Name field is empty.
Users cannot update genre if the Description field is empty.
Flow scenario:
User successfully updates a genre record.
User cannot update a genre record if required fields are left blank.
User receives incorrect data messages when entering invalid genre data.
User encounters an error message during the genre update process.
User views updated genre list after successful record update.
Users successfully update a genre record.
Users cannot update a genre record if entered data is invalid.
Users encounter an error while updating a genre record.
Users cancel the update operation before submitting the fo

In [7]:
promptExtractCondition = [
    { "role": "system", "content": EXTRACT_CONDITION_SYSTEM_PROMPT},
    { "role": "user", "content": prompt_all}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
full_elements = json.loads(gpt_response)
condition_element = {key: value for key, value in full_elements.items() 
                              if (value['type'] in ['text field','text area'] and (value['condition']['valid'] or value['condition']['invalid'] ))}
print(condition_element)

filtercontent = ""

if(len(condition_element)!= 0):
  promptScenarioForCondition = [
      { "role": "system", "content": GEN_SCENARIO_FOR_CONDITION_PROMPT},
      { "role": "user", "content": prompt_all + '/n Element:' + str(condition_element)}
    ]
  condition_scenario_response = ask(promptScenarioForCondition, client, model)
  print(condition_scenario_response)
  filtercontent =  "Conditional scenario:" + "\n" + condition_scenario_response


promptMainScenario = [
    { "role": "system", "content": MAIN_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": main_flow_prompt}
  ]
main_gpt_response = ask(promptMainScenario, client, model)

promptSubScenario1 = [
    { "role": "system", "content": SUB_FLOW_SYSTEM_PROMPT},
    { "role": "user", "content": alt_prompt_1}
  ]
sub1_gpt_response = ask(promptSubScenario1, client, model)

filtercontent +="\n" + "Flow scenario:" + "\n" + main_gpt_response + "\n" + sub1_gpt_response
print(filtercontent)
promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
print("\n" + "Final result:" + filter_gpt_response)

promptFilter = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT},
    { "role": "user", "content": filtercontent}
  ]
filter_gpt_response = ask(promptFilter, client, model)
print("\n" + "Final result:" + filter_gpt_response)

promptExtractCondition = [
    { "role": "system", "content": Prompt_Extract_Action},
    { "role": "user", "content": filter_gpt_response}
  ]
gpt_response = askJSON(promptExtractCondition, client, model)
print(gpt_response)

group_testscearnio = json.loads(gpt_response)

promptExtractCondition = [
    { "role": "system", "content": FILTER_SYSTEM_PROMPT3},
    { "role": "user", "content": str(group_testscearnio)}
  ]
gpt_response = ask(promptExtractCondition, client, model)
write_string_to_file(usecase_name + "-Filter3.csv", filter_gpt_response)
print(gpt_response)

{'Genre Name': {'condition': {'valid': 'user enters a name for the genre', 'invalid': 'name field is left empty'}, 'type': 'text field'}, 'Genre Description': {'condition': {'valid': 'user enters a description for the genre', 'invalid': 'description field is left empty'}, 'type': 'text area'}}
User cannot update genre if the Genre Name field is left empty.
User cannot update genre if the Genre Description field is left empty.
Conditional scenario:
User cannot update genre if the Genre Name field is left empty.
User cannot update genre if the Genre Description field is left empty.
Flow scenario:
User successfully updates a genre record.
User cannot update a genre record with incorrect data.
User encounters an error while updating a genre record.
User views updated genre list after successful update.
Users successfully update a genre with valid data.
Users cannot update a genre with invalid data.
Users cancel the update operation before submitting the form.
Users navigate away from the u